In [1]:
import pandas as pd
import numpy as np

/Users/ilenapeng/.pyenv/versions/3.9.7/lib/python3.9/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
df = pd.read_csv('carf-accredited-merged.csv')
df.head()

,name,link,city,state,company,type,address,address_pt2,phone,website,program_offered_1,program_offered_2,program_offered_3,program_offered_4,program_offered_5,program_offered_6,program_offered_7,program_offered_8,program_offered_9,program_offered_10
0,Anna Westin House,http://www.carf.org/providerProfile.aspx?cid=3...,Saint Paul,MN,"The Emily Program, LLC",BH,1449 Cleveland Avenue North,"Saint Paul, MN 55108",651-645-5323,651-645-5323,Residential Treatment (BH) Mental Health Eatin...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Anna Westin House Adolescent and Young Adult,http://www.carf.org/providerProfile.aspx?cid=3...,Saint Paul,MN,"The Emily Program, LLC","BH, children and adolescents",2230 Como Avenue,"Saint Paul, MN 55108",651-645-5323,651-645-5323,Residential Treatment (BH) Mental Health Eatin...,Residential Treatment (BH) Mental Health Eatin...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Anna Westin House West,http://www.carf.org/providerProfile.aspx?cid=3...,Minneapolis,MN,"The Emily Program, LLC",BH,3012 44th Street West,"Minneapolis, MN 55410",651-645-5323,651-645-5323,Residential Treatment (BH) Mental Health Eatin...,Residential Treatment (BH) Mental Health Eatin...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Appalachian Outpatient Services, LLC dba Tapes...",http://www.carf.org/providerProfile.aspx?cid=2...,Brevard,NC,Clearview Pyramid Acquisition Company - Southeast,"BH, children and adolescents",11 North Country Club,"Brevard, NC 28712",855-396-2604,855-396-2604,Intensive Outpatient Treatment (BH) Mental Hea...,Partial Hospitalization Mental Health Adults,Partial Hospitalization Mental Health Eating D...,Residential Treatment (BH) Mental Health Adults,Residential Treatment (BH) Mental Health Eatin...,NaN,NaN,NaN,NaN,NaN
4,"CRC ED Treatment, Inc. dba Montecatini",http://www.carf.org/providerProfile.aspx?cid=2...,Carlsbad,CA,Acadia Healthcare,BH,2524 La Costa Avenue,"Carlsbad, CA 92009",760-436-2657,760-436-2657,Residential Treatment (BH) Mental Health Eatin...,Residential Treatment (BH) Mental Health Eatin...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# create one massive column that we'll use to fill in our columns - fill nas with blanks so that the summed column doesn't return an NA
df['program_all'] = df.fillna('').program_offered_2 + ' / ' + df.fillna('').program_offered_3 + ' / ' + df.fillna('').program_offered_4 + ' / ' + \
       df.fillna('').program_offered_5 + ' / ' + df.fillna('').program_offered_6 + ' / ' + df.fillna('').program_offered_7 + ' / ' + \
       df.fillna('').program_offered_8 + ' / ' + df.fillna('').program_offered_9 + ' / ' + df.fillna('').program_offered_10

In [4]:
df = df.replace('Residential Treatment (BH) Mental Health Eating Disorders', 'Residential Treatment (BH) Mental Health Eating Disorders - Adults')

In [5]:
# create blank columns to fill
df[[
    # treatment types: hospital inpatient, residential, partial hospitalization day treatment, outpatient
    'hi', 'res', 'phdt', 'op',
    # facility types: 
    'cbhc', 'msmh', 'omh', 'ores', 'ph', 'psy', 'rtca', 'rtcc', 'ipsy', 'shp', 'vahc',
    # age groups: child (<12), young adult (18-25), adult (25-64), senior (65+) 
    'chld', 'yad', 'adlt', 'snr',
    # special programs offered - ped is EDs
    'tay', 'se', 'gl', 'vet', 'adm', 'mf', 'cj', 'co', 'hv', 'dv', 'trma', 'tbi', 'alz', 'ped', 'pefp', 'ptsd', 'sed', 'smi'
   ]] = None

df.columns

Index(['name', 'link', 'city', 'state', 'company', 'type', 'address',
       'address_pt2', 'phone', 'website', 'program_offered_1',
       'program_offered_2', 'program_offered_3', 'program_offered_4',
       'program_offered_5', 'program_offered_6', 'program_offered_7',
       'program_offered_8', 'program_offered_9', 'program_offered_10',
       'program_all', 'hi', 'res', 'phdt', 'op', 'cbhc', 'msmh', 'omh', 'ores',
       'ph', 'psy', 'rtca', 'rtcc', 'ipsy', 'shp', 'vahc', 'chld', 'yad',
       'adlt', 'snr', 'tay', 'se', 'gl', 'vet', 'adm', 'mf', 'cj', 'co', 'hv',
       'dv', 'trma', 'tbi', 'alz', 'ped', 'pefp', 'ptsd', 'sed', 'smi'],
      dtype='object')

In [6]:
# treatment types
# residential
df['res'] = np.where(df.program_all.str.contains('Residential Treatment'), 1, None)
# partial hospitalization / day treatment
df['phdt'] = np.where(df.program_all.str.contains('Partial Hospitalization|Day Treatment'), 1, None)
# outpatient
df['op'] = np.where(df.program_all.str.contains('Intensive Outpatient Treatment'), 1, None)

# facility types
# msmh - Multi-setting mental health facility (e.g., non-hospital residential plus either outpatient and/or partial hospitalization/day treatment)
df['msmh'] = np.where(df.program_all.str.contains('Intensive Outpatient Treatment|Partial Hospitalization') & (df.program_all.str.contains('Residential Treatment')), 1, None)
# ohm - outpatient mental health
df['omh'] = np.where(df.program_all.str.contains('Intensive Outpatient Treatment'), 1, None)
# ph - outpatient mental health
df['ph'] = np.where(df.program_all.str.contains('Partial Hospitalization'), 1, None)

# age groups
df['chld'] = np.where(df.program_all.str.contains('Residential Treatment (BH) Mental Health Eating Disorders - Children and Adolescents'), 1, None)
df['yad'] = np.where(df.program_all.str.contains('Residential Treatment (BH) Mental Health Eating Disorders'), 1, None)
df['adlt'] = np.where(df.program_all.str.contains('Residential Treatment (BH) Mental Health Eating Disorders'), 1, None)
df['snr'] = np.where(df.program_all.str.contains('Residential Treatment (BH) Mental Health Eating Disorders'), 1, None)

# treatment types
# eating disorders
df['ped'] = np.where(df.program_all.str.contains('Eating Disorders'), 1, None)
# co-occurring mental and substance use disorders
df['co'] = np.where(df.program_all.str.contains('SUD|Substance Use Disorders|'), 1, None)

/var/folders/9x/ttxn_gwj0592mr6h8s5y29980000gn/T/ipykernel_16082/2657621872.py:18: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  df['chld'] = np.where(df.program_all.str.contains('Residential Treatment (BH) Mental Health Eating Disorders - Children and Adolescents'), 1, None)
/var/folders/9x/ttxn_gwj0592mr6h8s5y29980000gn/T/ipykernel_16082/2657621872.py:19: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  df['yad'] = np.where(df.program_all.str.contains('Residential Treatment (BH) Mental Health Eating Disorders'), 1, None)
/var/folders/9x/ttxn_gwj0592mr6h8s5y29980000gn/T/ipykernel_16082/2657621872.py:20: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  df['adlt'] = np.where(df.program_all.str.contains('Residential Treatment (BH) Mental Health Eating Disorders'), 1, None)
/var/folders/9x/ttxn_gwj0592mr6h8s5y29980000gn/T/ipykernel_16082/2657621872.py:21: UserWa

In [7]:
# drop the columns
df = df.drop(columns=['program_offered_1',
       'program_offered_2', 'program_offered_3', 'program_offered_4',
       'program_offered_5', 'program_offered_6', 'program_offered_7',
       'program_offered_8', 'program_offered_9', 'program_offered_10',
       'program_all'])

In [8]:
# split addresses into columns
df['city'] = df['address_pt2'].str.split(', ', expand=True)[0]
df['state'] = df['address_pt2'].str.split(', ', expand=True)[1]

# split state column to get zip
df['zip'] = df['state'].str.split(' ', expand=True)[2]

In [9]:
# replace the state column with itself, but without the zip (could also use a regex replace here)
df['state'] = df['state'].str.split(' ', expand=True)[0]

In [10]:
df.to_csv('carf-accredited-for-concat.csv', index=False)